In [ ]:
!pip -q install langchain

# Custom LLM Chain

# Custom langchain conversation chain

In [ ]:
import json
import os
import re
import requests
from getpass import getpass
from pathlib import Path
import inspect
import langchain
from langchain.chains import ConversationChain, LLMChain, LLMMathChain, TransformChain, SequentialChain
from langchain.chat_models import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.llms.base import LLM, Optional, List, Mapping, Any
from langchain.embeddings.openai import OpenAIEmbeddings
from textwrap import dedent
from langchain.memory import (
    ChatMessageHistory,
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryBufferMemory,
    VectorStoreRetrieverMemory,
)
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import messages_from_dict, messages_to_dict
from langchain.vectorstores import Chroma
from langchain.agents import load_tools
from langchain.agents import initialize_agent
import json


# custom langchain buffer window memory

In [ ]:
from typing import Any, Dict, List
from langchain.memory.chat_memory import BaseChatMemory
from langchain.schema import BaseMessage


def get_buffer_string(
    messages: List[BaseMessage], human_prefix: str = "Human", ai_prefix: str = "AI"
) -> str:
   
    # Select all HumanMessages
    print(messages)
    """Get buffer string of messages."""
    string_messages = []
    for m in messages:
        if isinstance(m, HumanMessage):
            role = human_prefix
        elif isinstance(m, AIMessage):
            role = ai_prefix
        elif isinstance(m, SystemMessage):
            role = "System"
        elif isinstance(m, ChatMessage):
            role = m.role
        else:
            raise ValueError(f"Got unsupported message type: {m}")
        if role == human_prefix:
            string_messages.append(f"{m.content}")
        else:
            string_messages.append(f"{role}: {m.content}")
    return "\n".join(string_messages)



class CustomBufferWindowMemory(BaseChatMemory):
    """Buffer for storing conversation memory."""

    human_prefix: str = "Human"
    ai_prefix: str = "AI"
    memory_key: str = "history"  #: :meta private:
    k: int = 5

    @property
    def buffer(self) -> List[BaseMessage]:
        """String buffer of memory."""
        return self.chat_memory.messages

    @property
    def memory_variables(self) -> List[str]:
        """Will always return list of memory variables.

        :meta private:
        """
        return [self.memory_key]

    def add_input_only(self, input_str: str) -> None:
        """Add only a user message to the chat memory."""
        self.chat_memory.add_user_message(input_str)

    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, str]:
        """Return history buffer."""

        buffer: Any = self.buffer[-self.k * 2 :] if self.k > 0 else []
        if not self.return_messages:
            buffer = get_buffer_string(
                buffer,
                human_prefix=self.human_prefix,
                ai_prefix=self.ai_prefix,
            )
        return {self.memory_key: buffer}

In [ ]:
# Enter your Kobold api link
# Example: Kobold_api_url = "https://blahblalhabal.trycloudflare.com"

Kobold_api_url = "https://specially-toyota-heating-healthy.trycloudflare.com"

In [ ]:
class KoboldApiLLM(LLM):
    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(self, prompt: str, stop: Optional[List[str]]=None) -> str:
        # Prepare the JSON data
        data = {
            "prompt": prompt,
            "use_story": False,
            "use_authors_note": False,
            "use_world_info": False,
            "use_memory": False,
            "max_context_length": 1800,
            "max_length": 100,
            "rep_pen": 1.12,
            "rep_pen_range": 1024,
            "rep_pen_slope": 0.9,
            "temperature": 0.5,
            "tfs": 0.9,
            "top_p": 0.95,
            "top_k": 0.6,
            "typical": 1,
            "frmttriminc": True
        }

        # Add the stop sequences to the data if they are provided
        if stop is not None:
            data["stop_sequence"] = stop

        # Send a POST request to the Kobold API with the data
        response = requests.post(f"{Kobold_api_url}/api/v1/generate", json=data)

        # Raise an exception if the request failed
        response.raise_for_status()

        # Return the generated text
        return response.json()["results"][0]["text"].strip().replace("```", " ")

    
    def __call__(self, prompt: str, stop: Optional[List[str]]=None) -> str:
        return self._call(prompt, stop)

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {}

llm = KoboldApiLLM()





## You can now test out the custom LLM wrapper in the next cell


In [ ]:
# llm("hello I am")

In [ ]:
class Chatbot:
    def __init__(self, bot_name):
        self.stop_token = "</s>"
        self.histories = {}  # Initialize the history dictionary
        self.stop_sequence = ["### Response:", "Human:"]
        self.llm = KoboldApiLLM()
        
        self.char_name = bot_name
        self.char_persona = "The following is a casual friendly discord group chat between a chatbot and the members of a discord group. She talkative and provides lots of specific details from its context. If the Tensor does not know the answer to a question, it truthfully says it does not know."
        self.memory = ConversationBufferWindowMemory(ai_prefix=self.char_name)
        self.chain = ConversationChain(llm=llm, memory=self.memory, verbose=True)
        self.template = f"""The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.
        
Current conversation:
{{history}}
{{input}}
{self.char_name}:"""
        self.PROMPT = PromptTemplate(input_variables=["history", "input"], template=self.template)


        self.conversation = ConversationChain(
            prompt=self.PROMPT,
            llm=llm,
            verbose=True,
            memory=CustomBufferWindowMemory(k=10, ai_prefix=self.char_name),
        )


    def generate_response(self, message_content, name) -> None:

        self.add_stop_sequence(name)
        formatted_message = f"{name}: {message_content}"

        input_dict = {
            "input": formatted_message, 
            "stop": self.stop_sequence
        }
        response = self.conversation(input_dict)

        # response = self.conversation.generate_with_botname(self.bot_name, [formatted_message], stop=self.stop_sequence)


        return response["response"]


    def add_stop_sequence(self, name):
        if name not in self.stop_sequence:
            self.stop_sequence.append(name)
        return name

    def remove_char_name_from_string(self, string):
        prefix_to_remove = self.char_name + "@"

        if string.startswith(prefix_to_remove):
            return string[len(prefix_to_remove):].lstrip()  # lstrip() is used to remove any leading whitespace
        else:
            return string
        


In [ ]:
# from google.colab import drive
# drive.mount('/content/chardata')

In [ ]:
chatbot = Chatbot("Tensor")

In [ ]:
# chatbot.generate_response("What is my name?", "Austin")
# print(chatbot.memory["history"])

response = chatbot.generate_response("I want you to remember this number 1024. What is 2 + 2?", "Austin")
print(response)

In [ ]:
response = chatbot.generate_response("What is your name?", "Austin")
print(response)

In [ ]:
response = chatbot.generate_response("What number did I ask you to remember?", "Austin")
print(response)

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
print(response)
# chatbot.save_conversation("more more!", "Austin")
# print(chatbot.memory.buffer)

In [ ]:
chatbot.memory.load_memory_variables('chat_history')['chat_history']

In [ ]:
print(chatbot.memory.load_memory_variables('chat_history'))

In [ ]:
pip install llama-index

In [ ]:
bootychihuahua.jpg

In [ ]:
from pathlib import Path
from llama_index import download_loader

ImageCaptionReader = download_loader("ImageCaptionReader")

loader = ImageCaptionReader()
documents = loader.load_data(file=Path('./bootychihuahua.jpg'))

In [ ]:
from langchain.chains import ConversationChain, LLMChain, LLMMathChain, TransformChain, SequentialChain

# Initialize the ConversationChain with the desired language model and memory system
chain = CustomConversationChain(llm=llm, memory=CustomBufferWindowMemory(), verbose=True)

# Start the conversation
response = chain.run("Hi there!")
print(response)

In [ ]:
response = chain.run("Remember this number 1")
print(response)